### Строим запрос к нашему сервису

In [1]:
import requests
import urllib.request
import json

In [8]:
# Пример данных
data = (
    "Stustwo offers you the opportunity to be yourself, whilst delivering thi",
    "ustwo offers you the opportunity ble on salary and there is the potent to be yourself, whilst delivering the best.",
    "We are negotiable on salary and there is the potential for equity for the right candidate.",
    "We help teachers get safe &amp"
)

In [9]:
# формируем запрос
def send_json(x):
    description, company_profile, benefits, requirements = x
    #print(description, company_profile, benefits, requirements)
    body = {
        'description': description,
        'company_profile': company_profile,
        'benefits': benefits,
        'requirements': requirements
        }
    myurl = 'http://127.0.0.1:5000/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [10]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.0178050763452046


### Сделаем обработку массы запросов:

In [11]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

X_test.fillna('', inplace=True)

In [13]:
X_test[['description', 'company_profile', 'benefits', 'requirements']]

,description,company_profile,benefits,requirements
0,Stylect is a dynamic startup that helps helps ...,,We are negotiable on salary and there is the p...,We don’t care where you studied or what your G...
1,General Summary: Achieves maximum sales profit...,,Great Health and DentalFast Advancement Opport...,
2,"At ustwo™ you get to be yourself, whilst deliv...",ustwo offers you the opportunity to be yoursel...,,Skills• Experience interfacing directly with c...
3,About EDITDEDITD runs the world's biggest appa...,"We build software for fashion retailers, to he...",,Required Skills / Experience:Ability to analys...
4,As a Web Engineer at Runscope you'll be respon...,Runscope is building tools for developers work...,Be a part of an experienced team who have work...,Extensive front-end web experience (HTML/CSS/J...
...,...,...,...,...
5896,Fabrication and Printing Company in Long Islan...,,Excellent opportunity to learn exci...,- You must be proficient in Photoshop a...
5897,Serve as the primary lead and project manager ...,"Palerra, Inc. designed LORIC™ to protect an en...",What's In It For You?Competitive compensation ...,"5-6 years running, managing and operating mark..."
5898,"Jiffy, a world wide leader in mobile applicati...",Jiffy Worldwide is the parent company of the J...,,iOS Developer should be very familiar and expe...
5899,What our client needs…A Medical Director who i...,Human capital is usually the biggest asset and...,,What you need to have….Doctor of Medicine (MD)...


### Передаем запрос одиночный из наших данных и посмотрим на ответ

In [14]:
response = send_json(X_test[['description', 'company_profile', 'benefits', 'requirements']].iloc[0,:])

In [15]:
response

0.03830971111833946

Сделаем N запросов и оценим время

In [16]:
N = 50

In [17]:
%%time
predictions = X_test[
                     ['description', 'company_profile', 'benefits', 'requirements']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: total: 78.1 ms
Wall time: 222 ms


In [18]:
predictions.values[:5]

array([0.03830971, 0.05181088, 0.00373199, 0.00121891, 0.0011764 ])

### Посчитаем метрику

In [19]:
y_test.head()

,fraudulent
0,0
1,0
2,0
3,0
4,0


In [20]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.1353388165933873, F-Score=1.000, Precision=1.000, Recall=1.000


In [21]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

1.0